In [1]:
import numpy as np
import pandas as pd
import os, sys
import glob
import tensorflow as tf
from tensorflow import keras
from scipy.fftpack import fft,ifft
import matplotlib.pyplot as plt
## Imports: Keras Models
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda
from keras.models import Sequential, Model
from keras.layers import Cropping2D
from keras.layers.core import Dropout
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
import random
from sklearn.model_selection import train_test_split

### Imports: data preprocessing 
import csv
import numpy as np
import os
import sklearn

sf = 250.
# Path for category 1
young_task_path = "/home/birdfly/Data/task/task young/"
# Path for category 2
young_non_task_path = "/home/birdfly/Data/task/non task young/"

freq_from = 0
freq_to = 20

channels_count = 26

Using TensorFlow backend.


In [2]:
def read_directory(young_task_path, young_non_task_path):
    """ 
        read all of the file under task_path
        :param list task_path : path contains all of the files
        :category task 0 or non task 1
        return a list
    """
    # task_datas = []
    X_train = []
    y_train = []
    for sub_path in os.listdir(young_task_path):
        task_data_files_paths = glob.glob(os.path.join(young_task_path, sub_path) + "/*.csv")
        for task_data_files_path in task_data_files_paths:
            data = pd.read_csv(task_data_files_path, sep=',')
            data_req = transfer_time_to_freq(data)
            X_train.append(data_req)
            y_train.append(0)

    for sub_path in os.listdir(young_non_task_path):
        task_data_files_paths = glob.glob(os.path.join(young_non_task_path, sub_path) + "/*.csv")
        for task_data_files_path in task_data_files_paths:
            data = pd.read_csv(task_data_files_path, sep=',')
            data_req = transfer_time_to_freq(data)
            X_train.append(data_req)
            y_train.append(1)
            
    X_train = np.asarray(X_train)
    y_train = tf.keras.utils.to_categorical(y=y_train, num_classes=2)
    
    return X_train, y_train

In [3]:
def transfer_time_to_freq(data, freq_from = freq_from, freq_to = freq_to):
    freq_data = []
    for row_index in data:  
        fft_y=fft(data[row_index])
        T = 1/250.  # sampling interval 
        N = data[row_index].size
        # 1/T = frequency
        f = np.linspace(0, 1 / T, N)
        freq_data_row = np.abs(fft_y)[:N // 2] * 1 / N
        freq_data.extend(freq_data_row[freq_from:freq_to])
    
    freq_data_t = pd.DataFrame(freq_data).T.values.tolist()
    return freq_data

In [4]:
def run_model(x_train, y_train, x_test, y_test):
    """
        run deep learning network and save trained model
        :param func train_generator
        :param func validation_generator
    """
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense((freq_to - freq_from)*channels_count, activation='relu'),
        tf.keras.layers.Dense(chanel_count, activation='relu'),
        tf.keras.layers.Dropout(0.05),
        tf.keras.layers.Dense((freq_to - freq_from), activation='relu'),
        tf.keras.layers.Dropout(0.05),
        tf.keras.layers.Dense(2, activation='softmax')
    ])

    model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
    model.fit(x_train, y_train, epochs= 10)
    model.evaluate(x_test, y_test)
    
    model.save('model.h5')
    print('model.h5 has been saved.')

if __name__ == "__main__":
    x, y = read_directory(young_task_path, young_non_task_path)
    
    x_train, x_test, y_train, y_test = train_test_split( x, y, test_size=0.2, random_state=42)

    run_model(x_train, y_train, x_test, y_test)

In [ ]:
if name == "main": x, y = read_directory(young_task_path, young_non_task_path)

x_train, x_test, y_train, y_test = train_test_split( x, y, test_size=0.2, random_state=42)

run_model(x_train, y_train, x_test, y_test)